# OLS

OLS is the most common technique used for regression. This tutorial gives an overview of OLS as implemented in Python's statsmodels and sci-kit learn packages, while the corresponding R script contains the corresponding R code. The end of the tutorial covers the numerical approaches used to calculate OLS.  

## Pulling Data and Imports

In [1]:
import pandas as pd
import numpy as np
import scipy
from sklearn import linear_model
import statsmodels.api as sm

In [2]:
x = pd.read_csv('C:\\Users\\smcdo\\OneDrive\\Documents\\Model_Framework\\Benchmarks\\x_data.csv', index_col=0)
y = pd.read_csv('C:\\Users\\smcdo\\OneDrive\\Documents\\Model_Framework\\Benchmarks\\y_data.csv', index_col=0, squeeze=True)

## Scikit-Learn

Scikit-Learn provides a common API to many machine learning techniques, however it does not provide any of the common statistical tests or information usually associated with OLS, such as p-values. 

In [3]:
ols_model = linear_model.LinearRegression(fit_intercept=True)
ols_model.fit(x,y)
print('Coefficients: %s' % ols_model.coef_)
print('Intercept: %s' % ols_model.intercept_)
print('R-Squared: %s' % ols_model.score(x,y))

Coefficients: [ 0.17773187 -0.84289583 -1.30278712  0.10972621 -1.45880093]
Intercept: 0.0025082808475
R-Squared: 0.997889460657


## Statsmodels

Statsmodels documention is very weak and it is not well-maintained, however it does provide the level of detail typically expected when implementing OLS regression.

In [4]:
x_sm = sm.add_constant(x)
ols_model = sm.OLS(y,x_sm)
ols_fit = ols_model.fit()
print(ols_fit.summary())

                            OLS Regression Results                            
Dep. Variable:                      0   R-squared:                       0.998
Model:                            OLS   Adj. R-squared:                  0.998
Method:                 Least Squares   F-statistic:                 9.451e+05
Date:                Wed, 30 Nov 2016   Prob (F-statistic):               0.00
Time:                        19:44:58   Log-Likelihood:                 8933.2
No. Observations:               10000   AIC:                        -1.785e+04
Df Residuals:                    9994   BIC:                        -1.781e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
const          0.0025      0.002      1.022      0.3

### Model Framework Package

In [5]:
import Machine_Learning_Interface.ols_regression as os
model = os.OLSRegression(intercept=True, scale=False)
model.fit(x, y)
model.diagnostics()
print('Coefficients: %s' % model.coefs)

C:\Users\smcdo\Anaconda2\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Users\smcdo\Anaconda2\lib\site-packages\sklearn\learning_curve.py:23: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the functions are moved. This module will be removed in 0.20
  DeprecationWarning)


Coefficients: 0            0.177732
1           -0.842896
2           -1.302787
3            0.109726
4           -1.458801
intercept    0.002508
Name: coefficients, dtype: float64


#### Anova 

In [11]:
import patsy
x_sm.columns = ['const', '0', '1', '2', '3', '4']
x_dmat = patsy.dmatrix("const + 0 + 1 + 2 + 3 + 4", x_sm)
ols_model = sm.OLS(y,x_dmat)
ols_fit = ols_model.fit()
ols_fit.summary()

PatsyError: numbers besides '0' and '1' are only allowed with **
    const + 0 + 1 + 2 + 3 + 4
                    ^

In [ ]:
anova = sm.stats.anova_lm(ols_fit, type=2)
print(anova)

## Numerical Techniques for OLS

The solution to the OLS coefficients is given by: beta = (X'X)^(-1)X'y, assuming (X'X)^(-1) is invertible. However, directly calculating the inverse is not recommended when solving OLS. There are 3 major ways OLS is solved: Cholesky, QR decomposition, and SVD, in order of speed. However, QR and SVD both have favorable numerical properties which generally make them the preferred method of solving OLS. R's lm function uses QR decomposition, with a LINPACK Fortran underlying interpretation. Sci-kit learn uses the scipy SVD implementation of LAPACK.  

### Direct Inverse Solution

In [12]:
def inverse_ols(x_data, y_data):
    inner = np.dot(x_data.T,x_data)
    beta = np.dot(np.dot(np.linalg.inv(inner),x_data.T),y_data)
    return beta
%timeit inverse_ols(x,y)
beta = inverse_ols(x,y)
print('Coefficients: %s' % beta)

1000 loops, best of 3: 572 µs per loop
Coefficients: [ 0.17828252 -0.8425956  -1.30342579  0.10971421 -1.4585405 ]


### Cholesky Decomposition

Cholesky Decomposition is fast compared to other approaches for solving OLS, however it's condition number is the square of the condition number of X'X, and therefore it can experience precision issues, especially is there is near-linear dependence among the independent variables.

In [13]:
def cholesky_ols(x_data, y_data):
    inner = np.dot(x_data.T, x_data)
    rhs = np.dot(x_data.T, y_data)
    L = np.linalg.cholesky(inner)
    for_sub = scipy.linalg.solve_triangular(L, rhs, lower=True) #Forward substitution
    beta = scipy.linalg.solve_triangular(L.T, for_sub) #Back substitution
    return beta
%timeit cholesky_ols(x,y)
beta = cholesky_ols(x,y)
print('Coefficients: %s' % beta)

The slowest run took 135.98 times longer than the fastest. This could mean that an intermediate result is being cached.
1000 loops, best of 3: 634 µs per loop
Coefficients: [ 0.17828252 -0.8425956  -1.30342579  0.10971421 -1.4585405 ]


### QR Decomposition

QR decomposition is 2x as slow as cholesky decomposition for large n, m, but it is much more numerically stable and therefore appropriate if there is a chance that there might be near linear dependence between two of the independent variables.

In [14]:
def qr_ols(x_data, y_data):
    q, r = np.linalg.qr(x_data)
    rhs = np.dot(q.T, y_data)
    beta = scipy.linalg.solve_triangular(r, rhs)
    return beta
%timeit qr_ols(x,y)
beta = qr_ols(x,y)
print('Coefficients: %s' % beta)

The slowest run took 107.88 times longer than the fastest. This could mean that an intermediate result is being cached.
1000 loops, best of 3: 631 µs per loop
Coefficients: [ 0.17828252 -0.8425956  -1.30342579  0.10971421 -1.4585405 ]


### SVD

Slowest method, but most numerically stable, and can handle rank-deficent problems. 

In [15]:
def svd_ols(x_data, y_data):
    x_mat = np.matrix(x)
    y_mat = np.matrix(y)
    u, s, v = np.linalg.svd(x_mat)
    s_inv = np.diag(1/s)
    xtra_zeros = np.zeros([x_mat.shape[1], x_mat.shape[0]-x_mat.shape[1]])
    s_full = np.hstack((s_inv,xtra_zeros))
    p_inv = v.T*s_full*u.T
    beta = p_inv*y_mat.T    
    return beta
%timeit svd_ols(x,y)
beta = svd_ols(x,y)
print('Coefficients: %s' % beta)

1 loop, best of 3: 2.58 s per loop
Coefficients: [[ 0.17828252]
 [-0.8425956 ]
 [-1.30342579]
 [ 0.10971421]
 [-1.4585405 ]]
